In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1662


In [2]:
using PyFormattedStrings
import Plots as plt

## HO Basis and Spatial System

In [3]:
exact = 0.8247 #Zanghellini value for infinite basis

0.8247

In [4]:
l = 10
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, V);
println(f"HO Basis Ref : E = {reference_energy(system):.4f}, D = {reference_energy(system) - exact:.4f}")

HO Basis Ref : E = 1.3837, D = 0.5590


## HF

In [5]:
hf = setup_HF(system);
compute_ground_state!(hf, max_iter = 1000, tol = 1e-10)
println(f"HF  : E = {energy(hf):.4f}, D = {energy(hf) - exact:.4f}")

rhf = setup_RHF(system);
compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10)
println(f"RHF : E = {energy(rhf):.4f}, D = {energy(rhf) - exact:.4f}")

HF  : E = 0.8450, D = 0.0203
RHF : E = 1.1796, D = 0.3549


In [22]:
rhf_sys = System(rhf)
l = rhf_sys.l
C = rhf_sys.transform

all_zero = true
for i in 2:2:l
    for j in 1:2:l
        if C[i, j] != 0
            all_zero = false
        end
        if C[j, i] != 0
            all_zero = false
        end
        C[i, j] = 1
        C[j, i] = 1
    end
end

restricted = true
for i in 1:2:l
    for j in 1:2:l
        if C[i, j] != C[i+1, j+1]
            println("no!")
        end
        C[i+1, j+1] = -C[i, j]
    end
end

C

20×20 Matrix{Float64}:
  0.933557      1.0          -1.06551e-12  …  -3.54487e-14   1.0
  1.0          -0.933557      1.0              1.0           3.54487e-14
 -7.13654e-13   1.0          -0.871417        -0.0161273     1.0
  1.0           7.13654e-13   1.0              1.0           0.0161273
  0.348741      1.0           7.64116e-13      1.84745e-13   1.0
  1.0          -0.348741      1.0          …   1.0          -1.84745e-13
 -1.71175e-13   1.0          -0.476233         0.0902805     1.0
  1.0           1.71175e-13   1.0              1.0          -0.0902805
  0.0827379     1.0           2.95437e-13     -3.79482e-13   1.0
  1.0          -0.0827379     1.0              1.0           3.79482e-13
 -4.07184e-15   1.0          -0.117575     …  -0.257044      1.0
  1.0           4.07184e-15   1.0              1.0           0.257044
  0.00066849    1.0           2.36665e-14      4.70859e-13   1.0
  1.0          -0.00066849    1.0              1.0          -4.70859e-13
  2.25335e-15   1.

In [ ]:
println(hf.C[:,1][1:4])
println(hf.C[:,2][1:4])
println("These are not spin-anti-symmetric")

## CCD

In [ ]:
ccd = setup_CCD(system);
compute_ground_state!(ccd, max_iter = 1000, tol = 1e-10)
println(f"CCD    : E = {energy(ccd):.4f}, D = {energy(ccd) - exact:.4f}")

ccd = setup_CCD(System(rhf));
compute_ground_state!(ccd, max_iter = 1000, tol = 1e-10)
println(f"CCD+RHF: E = {energy(ccd):.4f}, D = {energy(ccd) - exact:.4f}")

ccd = setup_CCD(System(hf));
compute_ground_state!(ccd, max_iter = 1000, tol = 1e-10)
println(f"CCD+HF : E = {energy(ccd):.4f}, D = {energy(ccd) - exact:.4f}")

## CCSD

In [ ]:
ccsd = setup_CCSD(system);
compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10)
println(f"CCSD    : E = {energy(ccsd):.4f}, D = {energy(ccsd) - exact:.4f}")

ccsd = setup_CCSD(System(rhf));
compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10)
println(f"CCSD+RHF: E = {energy(ccsd):.4f}, D = {energy(ccsd) - exact:.4f}")

ccsd = setup_CCSD(System(hf));
compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10)
println(f"CCSD+HF : E = {energy(ccsd):.4f}, D = {energy(ccsd) - exact:.4f}")

In [ ]:
big_system = System(n, SpinBasis(HOBasis(30, ω)), grid, V);

ccsd = setup_CCSD(big_system);
compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10)
println(f"CCSD    : E = {energy(ccsd):.4f}, D = {energy(ccsd) - exact:.4f}")